# Rozpoznawanie jednostek nazewniczych (NER) — Zadania

## Zadanie

Dla wybranej strony z Wikipedii porównaj rozpoznane jednostki nazewnicze przy użyciu spaCy (gruboziarnisty model NKJP) i PolDeepNer2 (drobnoziarnisty model KPWr).

Porównanie powinno uwzględniać:
1. Dla 20 najczęstszych form tekstowych rozpoznanych jako jednostki nazewnicze przez spaCy wypisz kategorie oraz liczność ich rozpoznania przez spaCy oraz PolDeepNer2, np. :
```
Gdańsk  [spaCy] (61) placeName   [PDN2] (58) nam_loc_gpe_city, ( 1) nam_org_institution, ( 1) nam_org_group_team, ( 2) nam_fac_goe_stop
```
2. Dla jednostki, dla której PolDeepNer2 wskazał więcej niż jedną kategorię, wypisz kontekst wystąpienia jednostki zinterpretowanej jako dana kategoria. Np. dla powyższego przykładu wypisz konteksty, w których `Gdańsk` został uznany za `nam_fac_goe_stop`, tj.
```
sanitarne lądowisko [Gdańsk]-Zaspa, a rok później
i otwarto lądowisko [Gdańsk]-Szpital UCK.
```
3. Dla jednostki, która nie została znaleziona w wynikach PolDeepNer2 wypisz te jednostki, których częścią może być dana jednostka. Na przykład dla:
```
Europy               [spaCy] ( 4) geogName                   [PDN2] brak
```
Można uzyskać:
```
Mistrzostw Europy w Koszykówce Mężczyzn 2009 [('nam_eve_human_sport', 1)]
Mistrzostw Europy w Piłce Nożnej 2012 [('nam_eve_human_sport', 1)]
Mistrzostw Europy w Piłce Siatkowej Mężczyzn 2013 [('nam_eve_human_sport', 1)]
Mistrzostw Europy w Piłce Ręcznej Mężczyzn 2016 [('nam_eve_human_sport', 1)]
```

**Wskazówka**

Przykładowa storna z Wikipedii:

In [ ]:
url = 'https://pl.wikipedia.org/wiki/Gda%C5%84sk'

**Pobranie treści strony**

In [ ]:
! pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

page = requests.get('https://pl.wikipedia.org/wiki/Gda%C5%84sk')
soup = BeautifulSoup(page.content)

texts = []
clean_regex = re.compile(r"\[\d+\]") # Do usunięcia przypisów w postaci [1].

for p in soup.select("div#mw-content-text p"): 
  text = p.get_text().strip()
  text = clean_regex.sub("", text)
  if len(text) > 100:
    texts.append(text)

print(len(texts))
print(*texts[:10], sep="\n")

90
Gdańsk (kaszub. Gduńsk, niem. Danzig, łac. Gedanum, Dantiscum, nid. Danswijk) – miasto na prawach powiatu w północnej Polsce w województwie pomorskim, położone nad Morzem Bałtyckim u ujścia Motławy do Wisły nad Zatoką Gdańską. Centrum kulturalne, naukowe i gospodarcze oraz węzeł komunikacyjny północnej Polski, stolica województwa pomorskiego. Ośrodek gospodarki morskiej z dużym portem handlowym.
Gdańsk z 471 525 mieszkańcami zajmuje szóste miejsce w Polsce pod względem liczby ludności, a siódme miejsce pod względem powierzchni – 263,44 km². Ośrodek aglomeracji trójmiejskiej, nazywaną też gdańską, wraz z Gdynią i Sopotem tworzą Trójmiasto.
Jest to miasto o ponadtysiącletniej historii, którego tożsamość na przestrzeni wieków kształtowała się pod wpływem różnych kultur. Gdańsk był również największym miastem Rzeczypospolitej Obojga Narodów, miastem królewskim i hanzeatyckim, posiadał prawo do czynnego uczestnictwa w akcie wyboru króla, w XVI w. był najbogatszym w Rzeczypospolitej. Mias

## Rozwiązanie

### Przygotowanie

#### Przygotowanie spaCy

In [ ]:
! pip install spacy==2.3.2 -U
! wget -O - http://download.sgjp.pl/apt/sgjp.gpg.key|sudo apt-key add -
! sudo apt-add-repository http://download.sgjp.pl/apt/ubuntu
! sudo apt update
! sudo apt install libmorfeusz2
! sudo apt install python3-morfeusz2
! wget -P zasoby http://mozart.ipipan.waw.pl/~alina/zasoby/data/pl_spacy_model_morfeusz-0.1.3.tar.gz
! python -m pip install zasoby/pl_spacy_model_morfeusz-0.1.3.tar.gz
! python -m spacy link pl_spacy_model_morfeusz pl_spacy_model_morfeusz -f

import spacy
nlp = spacy.load("pl_spacy_model_morfeusz")

     |████████████████████████████████| 10.0MB 14.8MB/s 
     |████████████████████████████████| 2.1MB 44.3MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
--2021-01-07 06:45:49--  http://download.sgjp.pl/apt/sgjp.gpg.key
Resolving download.sgjp.pl (download.sgjp.pl)... 193.0.67.154
Connecting to download.sgjp.pl (download.sgjp.pl)|193.0.67.154|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1029 (1.0K) [application/pgp-keys]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.00K  --.-KB/s    in 0s      

2021-01-07 06:45:49 (207 MB/s) - written to stdout [1029/1029]

OK
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.d

#### Przygotowanie PolDeepNer2

In [ ]:
!pip install https://pypi.clarin-pl.eu/packages/poldeepner2-0.1.0-py3-none-any.whl#md5=333b5d5270c6b1f6cd50df5c4c89f4dd

!python -m spacy download pl_core_news_sm
!python -m spacy link pl_core_news_sm pl_core_news_sm -f

import poldeepner2.models
ner_kpwr = poldeepner2.models.load("kpwr-n82-base", device="cuda:0")

     |████████████████████████████████| 10.5MB 22.1MB/s 
     |████████████████████████████████| 307kB 50.0MB/s 
     |████████████████████████████████| 276kB 56.1MB/s 
     |████████████████████████████████| 1.4MB 49.9MB/s 
     |████████████████████████████████| 184kB 53.9MB/s 
     |████████████████████████████████| 71kB 11.1MB/s 
     |████████████████████████████████| 102kB 14.4MB/s 
     |████████████████████████████████| 890kB 42.4MB/s 
     |████████████████████████████████| 1.1MB 42.6MB/s 
     |████████████████████████████████| 7.2MB 46.5MB/s 
     |████████████████████████████████| 71kB 12.5MB/s 
  Created wheel for fairseq: filename=fairseq-0.9.0-cp36-cp36m-linux_x86_64.whl size=2064549 sha256=e4baa724083c9b27fd7a828e0a4e172235b9dddae857062f64f4650e20dece1d
  Stored in directory: /root/.cache/pip/wheels/37/3e/1b/0fa30695dcba41e4b0088067fa40f3328d1e8ee78c22cd4766
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434677 sha256=33ee4d2be5b0ae375a048352803dd35

     |████████████████████████████████| 48.7MB 1.2MB/s 
  Created wheel for pl-core-news-sm: filename=pl_core_news_sm-2.3.0-cp36-none-any.whl size=48719490 sha256=a54e4ef92327ce7bbb7c892dc574359f12b0ae0a5d7dfe95cbaf2d559af43776
  Stored in directory: /tmp/pip-ephem-wheel-cache-f8hft64h/wheels/ce/7a/53/aab09c53056da9fefa67b44d57202e9bd556825e12ca89fff1
Successfully built pl-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pl_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pl_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pl_core_news_sm
You can now load the model via spacy.load('pl_core_news_sm')


kpwr_n82_base.zip: 452MB [00:19, 23.4MB/s]                           
roberta_base_fairseq.zip: 231MB [00:03, 71.4MB/s]                           


loading archive file models/roberta_base_fairseq
| dictionary: 50000 types


### Rozpoznanie jednostek

#### spaCy (NKJP)

In [ ]:
names_spacy = []

for text in texts:
  names_spacy += nlp(text).ents

print(*names_spacy[:10], sep="\n")

Gdańsk
Gduńsk
niem.
Gedanum
Dantiscum
Polsce
województwie pomorskim
Morzem Bałtyckim
Motławy
Wisły


#### PolDeepNer2 (KPWr)

In [ ]:
sentence_kpwr_annotations = []
names_kpwr = []

for text in texts:
  names = ner_kpwr.process_text(text)
  names_kpwr += names
  sentence_kpwr_annotations.append((text, names))

print(*names_kpwr[:10], sep="\n")

AnnotationText(begin=0, end=6, label='nam_loc_gpe_city', text='Gdańsk')
AnnotationText(begin=16, end=22, label='nam_loc_gpe_city', text='Gduńsk')
AnnotationText(begin=30, end=36, label='nam_liv_person', text='Danzig')
AnnotationText(begin=68, end=76, label='nam_loc_gpe_city', text='Danswijk')
AnnotationText(begin=118, end=124, label='nam_loc_gpe_country', text='Polsce')
AnnotationText(begin=140, end=149, label='nam_loc_gpe_admin1', text='pomorskim')
AnnotationText(begin=164, end=180, label='nam_loc_hydronym_sea', text='Morzem Bałtyckim')
AnnotationText(begin=190, end=197, label='nam_loc_hydronym_river', text='Motławy')
AnnotationText(begin=201, end=206, label='nam_loc_hydronym_river', text='Wisły')
AnnotationText(begin=211, end=225, label='nam_loc_hydronym', text='Zatoką Gdańską')


### Porównanie wyników

#### Kategorie dla najczęstszych (wg. spaCy) form tekstowych 

In [ ]:
from nltk import FreqDist

# Najczęstsze nazwy wskazane przez spaCy
names_spacy_lexems = {}
for name in names_spacy:
  names_spacy_lexems.setdefault(name.text, FreqDist()).update([name.label_])

names_spacy_lexems_sorted = sorted(names_spacy_lexems.items(), key=lambda x: x[1].N(), reverse=True)
names_spacy_lexems_sorted[:20]

[('Gdańsk', FreqDist({'placeName': 61})),
 ('Gdańska', FreqDist({'placeName': 34})),
 ('Gdańsku', FreqDist({'placeName': 23})),
 ('Polsce', FreqDist({'placeName': 10})),
 ('Polski', FreqDist({'placeName': 10})),
 ('Wisły', FreqDist({'geogName': 7})),
 ('Motławy', FreqDist({'geogName': 5})),
 ('Sopot', FreqDist({'placeName': 5})),
 ('Gdynia', FreqDist({'placeName': 4})),
 ('Westerplatte', FreqDist({'geogName': 4})),
 ('Europy', FreqDist({'geogName': 4})),
 ('Gdanczk', FreqDist({'placeName': 3})),
 ('Gdansk', FreqDist({'placeName': 3})),
 ('polskim', FreqDist({'placeName': 3})),
 ('polskich', FreqDist({'placeName': 3})),
 ('Prus', FreqDist({'geogName': 1, 'placeName': 2})),
 ('Gdańskie', FreqDist({'placeName': 3})),
 ('gdańskiej', FreqDist({'placeName': 3})),
 ('Nowy Port', FreqDist({'placeName': 3})),
 ('XIX wieku', FreqDist({'date': 3}))]

In [ ]:
names_kpwr_lexems = {}

for name in names_kpwr:
  names_kpwr_lexems.setdefault(name.text, FreqDist()).update([name.label])

In [ ]:
# Porównanie kategorii spaCy i PolDeepNer2 (KPWr)

for name, spacy_categories in names_spacy_lexems_sorted[:20]:
  spacy_categories = ["(%2d) %s" % (count, name) for name, count in spacy_categories.items()]
  kpwr_categories = ["brak"]
  if name in names_kpwr_lexems:
    kpwr_categories = ["(%2d) %s" % (count, name) for name, count in names_kpwr_lexems.get(name).items()]

  print("%-20s [spaCy] %-30s  [PDN2] %s" % (name, ", ".join(spacy_categories), ", ".join(kpwr_categories)))

Gdańsk               [spaCy] (61) placeName                  [PDN2] (58) nam_loc_gpe_city, ( 1) nam_org_institution, ( 1) nam_org_group_team, ( 2) nam_fac_goe_stop
Gdańska              [spaCy] (34) placeName                  [PDN2] (28) nam_loc_gpe_city, ( 1) nam_adj_city, ( 1) nam_pro_award
Gdańsku              [spaCy] (23) placeName                  [PDN2] (28) nam_loc_gpe_city, ( 1) nam_fac_goe
Polsce               [spaCy] (10) placeName                  [PDN2] (10) nam_loc_gpe_country
Polski               [spaCy] (10) placeName                  [PDN2] (10) nam_loc_gpe_country
Wisły                [spaCy] ( 7) geogName                   [PDN2] ( 7) nam_loc_hydronym_river
Motławy              [spaCy] ( 5) geogName                   [PDN2] ( 4) nam_loc_hydronym_river, ( 1) nam_loc_hydronym_lake
Sopot                [spaCy] ( 5) placeName                  [PDN2] ( 4) nam_loc_gpe_city
Gdynia               [spaCy] ( 4) placeName                  [PDN2] ( 4) nam_loc_gpe_city
Westerplatte 

#### Wypisanie kontekstu

In [ ]:
for sentence, names in sentence_kpwr_annotations:
  names_filtered = [name for name in names if name.label == "nam_fac_goe_stop" and name.text == "Gdańsk"]
  for an in names_filtered:
    context_begin = an.begin-20
    context_begin = (" " + sentence[0:context_begin]).rindex(" ")
    context_begin = 0 if context_begin == 0 else context_begin - 1

    context_end = an.end+20
    context_end += (sentence[context_end:-1] + " ").index(" ")

    print(sentence[context_begin:an.begin] + "[" + sentence[an.begin:an.end] + "]" + sentence[an.end:context_end])

 sanitarne lądowisko [Gdańsk]-Zaspa, a rok później
 Dębinki otwarto lądowisko [Gdańsk]-Szpital UCK.


#### Zawieranie się jednostek

In [ ]:
for name, freqs in names_kpwr_lexems.items():
  if "Europy" in name and name not in names_spacy_lexems:
    print(f"{name:<20} {freqs.most_common(5)}")

Mistrzostw Europy w Koszykówce Mężczyzn 2009 [('nam_eve_human_sport', 1)]
Mistrzostw Europy w Piłce Nożnej 2012 [('nam_eve_human_sport', 1)]
Mistrzostw Europy w Piłce Siatkowej Mężczyzn 2013 [('nam_eve_human_sport', 1)]
Mistrzostw Europy w Piłce Ręcznej Mężczyzn 2016 [('nam_eve_human_sport', 1)]


In [ ]:
for name, freqs in names_kpwr_lexems.items():
  if "Gdańskie" in name and name not in names_spacy_lexems:
    print(f"{name:<20} {freqs.most_common(5)}")

Konstytucje Gdańskie [('nam_pro_title_document', 1)]
Wojny Rzeczypospolitej z Gdańskiem [('nam_eve_human', 1)]
Międzynarodowe Targi Gdańskie [('nam_eve_human', 1)]
Gdańskie Autobusy i Tramwaje [('nam_org_company', 1)]
Gdańskie Noce Jazsowe [('nam_eve_human_cultural', 1)]
